# Building a Custom Costing Model in WaterTAP

## Overview

This tutorial demonstrates how to build a costing model in WaterTAP for our custom-made filtration unit as an example.

### Key Steps:

1. **Import Required Utilities**: Import helper functions from `watertap.costing.util` for registering parameters and creating cost variables

2. **Create the Global Parameters Function**: Create a function to define unit-specific global parameters such as:
   - Capital cost coefficients (base and exponent)
   - Fixed operating cost fractions
   - Electricity intensity
   - Other unit-specific parameters

3. **Create the Unit Costing Function**: Develop the main costing function that:
   - Creates capital and fixed operating cost variables
   - Adds cost factors (TIC or TPEC)
   - Defines constraints for calculating costs
   - Registers cost flows (e.g., electricity)

4. **Register the Global Parameters to the Unit Model Costing Model**: Register the global parameters to the costing function

By the end of this tutorial, you'll understand the structure and components needed to create a custom costing model that integrates seamlessly with WaterTAP's costing framework.

# Structure of the Cost Model File

#### The costing model usually has two key components or functions:
1. Function to define the global parameters for a unit model
2. The cost_model function - Unit models are mapped to this function

# Step 1: Import Required Utilities

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
import pyomo.environ as pyo

from watertap.costing.util import (
    register_costing_parameter_block,   # Used to register the global costing parameters
    make_capital_cost_var,              # Used to create capital cost variables
    make_fixed_operating_cost_var,      # Used to create fixed operating cost variables
)

## Step 2: Create the Global Parameters Function

The naming convention usually followed is `build_unit_model_name_cost_param_block`.

Here the `unit_model_name` can be replaced by the name of the specific unit process as preferred. In this example,`filtration` is used.

In this function, global parameters that are used to calculate the capital, fixed operating costs and flow costs can be defined. In this example:
1. `capital_cost_eq_base`: Capital cost coefficient A for custom filtration
2. `capital_cost_eq_exponent`: Capital cost exponent b for custom filtration
3. `fixed_op_cost_frac`: Fixed operating cost as a fraction of capital cost for custom filtration
4. `electricity_intensity`: Electricity intensity for custom filtration

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def build_filtration_cost_param_block(blk):
    """
    Create a costing parameter block for the custom filtration unit model.
    """

    blk.capital_cost_eq_base = pyo.Var(
        initialize=0.72557e6,
        units=pyo.units.USD_2014,
        doc="Capital cost coefficient A for custom filtration",
    )
    blk.capital_cost_eq_exponent = pyo.Var(
        initialize=0.5862,
        units=pyo.units.dimensionless,
        doc="Capital cost exponent b for custom filtration",
    )
    blk.fixed_op_cost_frac = pyo.Var(
        initialize=0.05,
        units=pyo.units.year**-1,
        doc="Fixed operating cost as a fraction of capital cost for custom filtration",
    )
    blk.electricity_intensity = pyo.Var(
        initialize=0.1,
        units=pyo.units.kWh / pyo.units.m**3,
        doc="Electricity intensity for custom filtration",
    )

**NOTE:** The global parameters created are registered to the cost model as shown in Step 4.

This way whenever an instance of the unit model is used in the flowsheet, the relevant global parameters are added to the flowsheet level costing block.

## Step 3: Create the Unit Costing Function

The purpose of this function is to: 
1. Create and define the capital and fixed operating costs variables 
2. Add any flows (such as electricity) to the costing model

The cost_model function is the default cost model that is used by a unit model. 

The naming convention usually followed is `cost_unit_model_name` where `unit_model_name` can be replaced by the name of the specific unit process. In this example, it is replaced by `filtration`.

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def cost_filtration(blk):
    """
    Costing method for Custom Filtration unit model.
    """

### Creating Capital and Fixed Operating Cost Variables

The following built-in functions are called within the cost model and create the following cost components as needed.

1. Capital Cost - `make_capital_cost_var`
2. Fixed Operating Cost - `make_fixed_operating_cost_var`

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def cost_filtration(blk):
    """
    Costing method for Custom Filtration unit model.
    """

    # Capital cost
    make_capital_cost_var(blk)


    # Fixed operating cost
    make_fixed_operating_cost_var(blk)
```    

Next, relevant cost factors and constraints to calculate the capital and fixed operating costs are defined.

* The global cost paramaters can be accessed in this function using `blk.costing_package.unit_model_name`. In this example, the `unit_model_name` is `filtration`.

* In the code below, notice:
    - The `add_cost_factor` function can be used to add the required cost factor. Valid strings for `factor` are "TIC" and "TPEC". 
    - It adds an expression pointing to the appropriate indirect capital cost multiplier, which is a expression defined to be `blk.capital_cost / blk.cost_factor`. ; all others will result in an indirect capital cost factor of 1.

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def cost_filtration(blk):
    """
    Costing method for Custom Filtration unit model.
    """

    # Capital cost
    make_capital_cost_var(blk)

    # Fixed operating cost
    make_fixed_operating_cost_var(blk)

    # Temporary variable for flow in MGD
    flow_vol_MGD = pyo.units.convert(
        blk.unit_model.properties_in[0].flow_vol_phase["Liq"]
        / (pyo.units.Mgallons / pyo.units.day),
        to_units=pyo.units.dimensionless,
    )

    # Get filtration global parameters from flowsheet level costing block
    filtration_params = blk.costing_package.filtration

    # Add cost factor
    blk.costing_package.add_cost_factor(blk, "TPEC")

In this example:
- The capital cost is calculated as:

$$\text{Capital Cost} = \text{cost\_factor} \times \left(\text{capital\_cost\_eq\_base} \times \text{flow\_vol\_MGD}^{\text{capital\_cost\_eq\_exponent}}\right)$$

- The fixed operating cost is calculated as:

$$\text{Fixed Operating Cost} = \text{fixed\_op\_cost\_frac} \times \text{Capital Cost}$$

```python 
    # Constraint to calculate capital cost
    blk.capital_cost_constraint = pyo.Constraint(
        expr=blk.capital_cost
        == blk.cost_factor
        * pyo.units.convert(
            filtration_params.capital_cost_eq_base
            * flow_vol_MGD**filtration_params.capital_cost_eq_exponent,
            to_units=blk.costing_package.base_currency,
        )
    )

    # Constraint to calculate the operating cost
    blk.fixed_operating_cost_constraint = pyo.Constraint(
        expr=blk.fixed_operating_cost
        == pyo.units.convert(
            filtration_params.fixed_op_cost_frac * blk.capital_cost,
            to_units=blk.costing_package.base_currency
            / blk.costing_package.base_period,

        )
    )

Any cost flows to be defined can be done as shown below within the cost_model function.

- An `Expression` is used in this example, to calculate the `blk.power_required` as a product of:
    - The global parameter `electricity_intensity` 
    - The volumetric flowrate in the unit model `blk.unit_model.properties_in[0].flow_vol_phase["Liq"]`

- The calculated value for `blk.power_required` is then used to add a "flow" variable called "electricity" to the flowsheet level costing block using `blk.costing_package.cost_flow`

- Notice that `blk.costing_package.cost_flow` requires two arguments:
    1. The variable object where we can find the value for flow term. In this example, the variable object is `blk.power_required`
    2. The name of the "flow". In this example, `electricity`

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
    blk.power_required = pyo.Expression(
            expr=pyo.units.convert(
             blk.unit_model.properties_in[0].flow_vol_phase["Liq"]
                * filtration_params.electricity_intensity,
                to_units=pyo.units.kW,
            )
        )

    blk.costing_package.cost_flow(blk.power_required, "electricity")
```

## Step 4: Register the Global Parameters to the Unit Costing Model

The global parameters created using `build_filtration_cost_param_block(blk)` should be registered to the costing function using the `register_costing_parameter_block` function.

* Notice the '@' before the function `register_costing_parameter_block`.

* The `register_costing_parameter_block` function has two required arguments or inputs:
    1. `build_rule`: This is the name of the function that creates the global parameters. 
    2. `parameter_block_name`: This defines the name the unit model block that be added to the flowsheet costing block (`m.fs.costing`)

In our example, when the `UnitModelCostingBlock` is called, a block named `filtration` will be added to the flowsheet costing block `m.fs.costing`.

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
@register_costing_parameter_block(
    build_rule=build_filtration_cost_param_block, parameter_block_name="filtration"
)
def cost_filtration(blk):
    """
    Costing method for Custom Filtration unit model.
    """

## Key Takeaways
- The costing model has two main components: global parameters and the unit costing function
- Global parameters are defined once and reused across all instances of the unit model
- `@register_costing_parameter_block` decorator should be used to link the global parameters function to the costing function
- The costing function defines how capital and operating costs are calculated
- Cost flows (like electricity) can be tracked at the flowsheet level
